In [9]:
import hls4ml
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects
import tensorflow as tf 
from util import create_config

root_dir  = f'models/deeppicar-stats/models/1011-000_64x64x1_0.1'
model_name = f'2-1011-000-0.1-100-30-50-70.h5'
model_name = f'{root_dir}/{model_name}'

co = {}
_add_supported_quantized_objects(co)
model = load_model(model_name, custom_objects=co)

## Print Model Summary

In [10]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 q_activation_28 (QActivatio  (None, 64, 64, 1)        0         
 n)                                                              
                                                                 
 conv1 (QConv2D)             (None, 30, 30, 2)         52        
                                                                 
 q_activation_29 (QActivatio  (None, 30, 30, 2)        0         
 n)                                                              
                                                                 
 q_conv2d_32 (QConv2D)       (None, 13, 13, 4)         204       
                                                                 
 q_activation_30 (QActivatio  (None, 13, 13, 4)        0         
 n)                                                              
                                                      

In [11]:
### Create HLS Model
directory = './tmp/tmp_prj'
config, hls_model = create_config(model, output_dir=directory)
print(config)

Interpreting Sequential
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 64, 64, 1]], output shape: [None, 64, 64, 1]
Layer name: q_activation_28, layer type: Activation, input shapes: [[None, 64, 64, 1]], output shape: [None, 64, 64, 1]
Layer name: conv1, layer type: QConv2D, input shapes: [[None, 64, 64, 1]], output shape: [None, 30, 30, 2]
Layer name: q_activation_29, layer type: Activation, input shapes: [[None, 30, 30, 2]], output shape: [None, 30, 30, 2]
Layer name: q_conv2d_32, layer type: QConv2D, input shapes: [[None, 30, 30, 2]], output shape: [None, 13, 13, 4]
Layer name: q_activation_30, layer type: Activation, input shapes: [[None, 13, 13, 4]], output shape: [None, 13, 13, 4]
Layer name: q_conv2d_34, layer type: QConv2D, input shapes: [[None, 13, 13, 4]], output shape: [None, 11, 11, 6]
Layer name: q_activation_31, layer type: Activation, input shapes: [[None, 11, 11, 6]], output shape: [None, 11, 11, 6]
Layer name: q_conv2d_35, layer type: QCon

In [12]:
hls_model.compile()

Writing HLS project
Done


In [13]:
### Check to see if it has the similar error
from get_dataset import get_dataset
from util import calculate_accuracy
import numpy as np

imgs_train, imgs_test, vals_train, vals_test = get_dataset('./deeppicar')

preds = hls_model.predict(imgs_test.astype(np.float32))
vals_test = np.array(vals_test)

# mse
print(f'HLS MSE: {np.mean(((preds.reshape(-1) - vals_test.reshape(-1)) ** 2))}')
print(f'Accuracy: {calculate_accuracy(preds, vals_test)}')

preds = model.predict(imgs_test.astype(np.float32))
vals_test = np.array(vals_test)

# mse
print(f'QKeras MSE: {np.mean(((preds.reshape(-1) - vals_test.reshape(-1)) ** 2))}')
print(f'Accuracy: {calculate_accuracy(preds, vals_test)}')

HLS MSE: 0.10809890180826187
Accuracy: 0.7604
79/79 [==============================] - 0s 3ms/step
QKeras MSE: 0.03689606487751007
Accuracy: 0.8472


## Build Accelerator

In [14]:
hls_model.build(csim=False)


****** Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2024.2 (64-bit)
  **** SW Build 5238294 on Nov  8 2024
  **** IP Build 5239520 on Sun Nov 10 16:12:51 MST 2024
  **** SharedData Build 5239561 on Fri Nov 08 14:39:27 MST 2024
  **** Start of session at: Sun May  4 12:29:19 2025
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2024 Advanced Micro Devices, Inc. All Rights Reserved.

source /opt/Xilinx/Vitis/2024.2/scripts/vitis_hls/hls.tcl -notrace
INFO: [HLS 200-10] For user 'petertso' on host 'pop-os' (Linux_x86_64 version 6.9.3-76060903-generic) on Sun May 04 12:29:19 CDT 2025
INFO: [HLS 200-10] On os Pop!_OS 22.04 LTS
INFO: [HLS 200-10] In directory '/home/petertso/Documents/753-FPGA/deepfpgamicro/tmp/tmp_prj'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-1510] Running: open_project myproject_prj 
INFO: [HLS 200-10] Opening project '/home/petertso/Documents/753-FPGA/deepfpgamicro/tmp/tmp_prj/myproject_prj'.
INFO: [HLS 200-1510] Run

{'CSynthesisReport': {'TargetClockPeriod': '30.00',
  'EstimatedClockPeriod': '21.039',
  'BestLatency': '225244',
  'WorstLatency': '225312',
  'IntervalMin': '16386',
  'IntervalMax': '225282',
  'BRAM_18K': '37',
  'DSP': '19',
  'FF': '14158',
  'LUT': '28217',
  'URAM': '0',
  'AvailableBRAM_18K': '100',
  'AvailableDSP': '90',
  'AvailableFF': '41600',
  'AvailableLUT': '20800',
  'AvailableURAM': '0'}}

## Read Vitis Report

In [15]:
hls4ml.report.read_vivado_report('tmp/tmp_prj')

Found 1 solution(s) in tmp/tmp_prj/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
SYNTHESIS REPORT:
== Vitis HLS Report for 'myproject'
* Date:           Sun May  4 12:29:41 2025

* Version:        2024.2 (Build 5238294 on Nov  8 2024)
* Project:        myproject_prj
* Solution:       solution1 (Vivado IP Flow Target)
* Product family: artix7
* Target device:  xc7a35t-cpg236-1


== Performance Estimates
+ Timing: 
    * Summary: 
    +--------+----------+-----------+------------+
    |  Clock |  Target  | Estimated | Uncertainty|
    +--------+----------+-----------+------------+
    |ap_clk  |  30.00 ns|  21.039 ns|     8.10 ns|
    +--------+----------+-----------+------------+

+ Latency: 
    * Summary: 
    +---------+---------+----------+----------+-------+--------+----------+
    |  Latency (cycles) |  Latency (absolute) |    Interval    | Pipeline |
    |   min   |   max   |    min   |    max   |  min  |   max  |   Type   |
    +---------+-----